# SOAP Note LLM Judge Comparison

This notebook demonstrates how to compare different LLMs used as judges and log the results to the Sagemaker-managed MLflow tracking server. It shows how to load the data, define a model that generates the SOAP notes, compose a set of LLM-as-a-Judge metrics, and define a judge models themselves.  
In order to run this notebook, make sure that the appropriate MLflow tracking server is running inside the Sagemaker Studio.

You can set the tracking server by name using the `set_sagemaker_tracking_server()` helper function. Next, specify the active experiment. The `set_experiment()` function creates the experiment with the provided name, if it does not exist yet.  
It is also recommended to use [tags](https://mlflow.org/docs/latest/getting-started/logging-first-model/step3-create-experiment#notes-on-tags-vs-experiments), which allow to filter the experiments. Every prompt engineering workflow must set a `task` tag to `prompt-engineering` and specify the descriptive project name in the `project` tag.

In [1]:
import os

import pandas as pd
from IPython.display import display

from utils.aws import (
    SAGEMAKER_DEFAULT_BUCKET,
    bedrock_runtime_client,
    sagemaker_session,
)
from utils.bedrock import BedrockModel
from utils.evaluation import compare_judge_models
from utils._mlflow import set_experiment, set_sagemaker_tracking_server
from utils.metrics.soap_note import completeness, source_grounding
from utils.prompts.soap_note import format_soap_note


DATA_DIR = "dataset"

set_sagemaker_tracking_server("a360-mlflow-tracking-server")
mlflow_experiment_name = "SOAP Note LLM-Judge Comparison"
mlflow_experiment_tags = {
    "task": "prompt-engineering",
    "project": "soap-note-generation",
    "mlflow.note.content": "This experiment compares different LLM judges for SOAP note evaluation",
}
experiment = set_experiment(mlflow_experiment_name, mlflow_experiment_tags)

/home/sagemaker-user/.conda/envs/soap-notes/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Data Preparation

See the `01-Data Pre-Processing.ipynb` notebook for instructions on how to prepare the data to be in compatible format

In [2]:
# load evaluation data
eval_data_path = os.path.join(DATA_DIR, "transcripts-plain.csv")
if not os.path.isfile(eval_data_path):
    sagemaker_session.download_data(
        DATA_DIR,
        SAGEMAKER_DEFAULT_BUCKET,
        "prompt-engineering/soap-notes/dataset/transcripts-plain.csv",
    )
eval_df = pd.read_csv(eval_data_path)
eval_df

,transcript
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me..."
1,Good morning. I'm Doctor. Chen. You must be Da...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett...."
3,"Hello, Jasmine. How have you been? Look, doc. ..."
4,"Hello, I'm Doctor. Patterson. You must be Jenn..."
5,Good morning. I'm Doctor. Chin. You must be Ka...
6,"Hello Mia, I'm Doctor. Harrison. Please come i..."
7,"Good morning, Mrs. Parker. I'm Doctor. Roberts..."
8,"Morning Ms. Davis, I'm Doctor. Warren. What br..."
9,Good morning Ms. Wright. How are you today? Gr...


## Model and Prompt Definition

Next, we need to define the model and prompt with which the outputs will be generated (SOAP notes in this case).  
Since the model will be invoked on each row of the input `DataFrame`, the prompt can specify variables (enclosed in single curly braces), whose names must correspond to column names in the input `DataFrame`.

In [3]:
SOAP_NOTE_GENERATION_PROMPT = """\
You are tasked with generating a SOAP note based on a transcript of a conversation between a doctor and a patient. A SOAP note is a method of documentation used by healthcare providers that includes four sections: Subjective, Objective, Assessment, and Plan.

Here is the transcript of the doctor-patient conversation:

<transcript>
{transcript}
</transcript>

Carefully analyze the transcript and extract relevant information for each section of the SOAP note. Follow these guidelines for each section:

1. Subjective: Include the patient's chief complaint, history of present illness, and any relevant past medical history, family history, or social history mentioned by the patient.

2. Objective: Note any physical examination findings, vital signs, or test results mentioned by the doctor. If specific measurements or results are not provided, leave this section brief or note that information was not available in the transcript.

3. Assessment: Summarize the doctor's diagnosis or differential diagnoses based on the subjective and objective information. Include any medical reasoning or thought process expressed by the doctor.

4. Plan: List any treatment plans, medications prescribed, further tests ordered, referrals made, or follow-up instructions given by the doctor.

Provide your output as a JSON object with four keys: "subjective", "objective", "assessment", and "plan". Each key should contain a string value summarizing the relevant information for that section."""
ASSISTANT_RESPONSE_PREFILL = """\
{
    "subjective": \""""

To create the model, you use the `BedrockModel` class. You must provide the Bedrock model ID, and the `bedrock-runtime` `boto3` client. Note that both model ID and [inference profile](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-profiles.html) ID are supported. You should prefer the inference profile whenever possible.  
Additionaly, you can provide the `inference_config` and `additional_req_fields` parameters, that correspond to `inferenceConfig` and `additionalmodelRequestFields` parameters of the Bedrock [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-call.html) respectively.

The `BedrockModel` class exposes a set of useful properties that are shown below.

In [4]:
haiku = BedrockModel(
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    bedrock_runtime_client,
    inf_config={"temperature": 0.0},
)
print(f"{haiku.id=}")
print(f"{haiku.info=}")
print(f"{haiku.name=}")
print(f"{haiku.inf_config=}")
print(f"{haiku.is_reasoner=}")

haiku.id='us.anthropic.claude-3-5-haiku-20241022-v1:0'
haiku.info={'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-5-haiku-20241022-v1:0', 'modelId': 'anthropic.claude-3-5-haiku-20241022-v1:0', 'modelName': 'Claude 3.5 Haiku', 'providerName': 'Anthropic', 'inputModalities': ['TEXT', 'IMAGE'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['INFERENCE_PROFILE'], 'modelLifecycle': {'status': 'ACTIVE'}}
haiku.name='claude-3.5-haiku'
haiku.inf_config={'temperature': 0.0}
haiku.is_reasoner=False


Next, you need to create an *evaluation function*. This is the function that MLflow will invoke with the input `DataFrame`. It is responsible for invoking the model on each row of this `DataFrame` and returning another `DataFrame` with all the information required to calculate the metrics.

The `BedrockModel` automates all of that by providing the `make_eval_fn()` helper method. Aside from the prompt, you can optionally specify the prefill message for the LLM (useful for generating JSON data) and a `custom_cols` parameter. This must be a dictionary that maps custom column names in the output `DataFrame` to a function that will be invoked with the raw Converse API response and return something that can be stored in the `DataFrame`, so you can use it to augment the output `DataFrame` with whatever you want.  
In the example below, `custom_cols` specifies the `soap_note` column and a `format_soap_note` function. This function transforms the JSON-formatted SOAP note output by an LLM into a plaintext, which later on will be passed to the Judge LLM.

`make_eval_fn()` also captures token usage and response latencies that can also be logged to MLflow as metrics.

In [5]:
PRED_COL_NAME = "soap_note"
haiku_eval_fn = haiku.make_eval_fn(
    SOAP_NOTE_GENERATION_PROMPT,
    ASSISTANT_RESPONSE_PREFILL,
    custom_cols={PRED_COL_NAME: format_soap_note},
)

## LLM Judge and Metrics Definition

LLM judges are simply defined as other `BedrockModel` instances. This example also shows how to define `BedrockModel`'s with reasoning capabilities

In [6]:
inf_config = {"temperature": 0.0}
sonnet = BedrockModel(
    "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    bedrock_runtime_client,
    inf_config=inf_config,
)
sonnet_thinking = BedrockModel(
    "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    bedrock_runtime_client,
    "claude-3.7-sonnet-thinking",
    # temperature is not supported for Sonnet 3.7 with extended reasoning
    {"maxTokens": 4096},
    {"thinking": {"type": "enabled", "budget_tokens": 2048}}
)
r1 = BedrockModel(
    "us.deepseek.r1-v1:0", bedrock_runtime_client, inf_config=inf_config
)
judge_models = [haiku, sonnet, sonnet_thinking, r1]

LLM-as-a-Judge metrics must be defined separately for each task. See the `utils/metrics/soap_note.py` file for an example of how LLM-as-a-Judge metrics can be defined.  
To create the metrics, we simply need to provide the list of aggregations we want to use to aggreate the per-row scores.

`completeness` metric measures the extent to which the SOAP note captures all the important medical information present in the original transcript.

`source_grounding` metric measures the extent to which the SOAP note includes only information that is directly supported by the transcript.

In [7]:
aggregations = ["min", "max", "mean", "median"]
judge_metrics = [completeness(aggregations), source_grounding(aggregations)]

## Evaluation
Finally, to run the comparison of different judges, you call the `compare_judge_models()` function. This function initiates the parent MLflow run, under which it logs nested runs that correspond to evaluations with individual judges.  
The function makes sure to generate the predictions only once, and then reuse these predictions across the judges to save the costs and ensure identical evaluation conditions. It also takes care of logging all the metics, judge model parameters and prompts that were used to calculate the metrics.

Apart from arguments that were discussed above, you must provide the name of the column in the `DataFrame` returned by the evaluation model which contains the predictions. Values from this column are then used to pass predictions (formatted SOAP notes in our case) to the metrics alongside the input transcripts.

In [8]:
eval_results = compare_judge_models(
    judge_models,
    haiku_eval_fn,
    eval_df,
    judge_metrics,
    PRED_COL_NAME,
    parent_run_name=haiku.name,
)

2025/04/16 12:09:34 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/04/16 12:09:34 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/04/16 12:12:33 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run claude-3.5-haiku at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34/runs/1dd082ddb89241ceb13467d1f3e7c4e5
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34


2025/04/16 12:18:46 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run claude-3.7-sonnet at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34/runs/dda636f743b94f3193242424e66d090b
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34


2025/04/16 12:28:03 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run claude-3.7-sonnet-thinking at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34/runs/9642eeae7ab44a4e95293fae893f52b4
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34


2025/04/16 12:54:35 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...


🏃 View run deepseek-r1 at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34/runs/3c9023eb026b4f0897e9a5093c56081e
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34
🏃 View run claude-3.5-haiku at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34/runs/30b4171901ae4947974e31c3054efba5
🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/34


`compare_judge_models` returns the dictionary that maps child run names to `mlflow.models.EvaluationResult` objects that can be further inspected to see the evaluation results for a particular run.

More detailed information can be seen, obviously, in the MLflow UI.

In [9]:
for run_name, result in eval_results.items():
    print(f"Evaluation results for run {run_name}:")
    display(result.tables["eval_results_table"].head(3))

Evaluation results for run claude-3.5-haiku:


,transcript,outputs,completeness/score,completeness/justification,source_grounding/score,source_grounding/justification
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me...",Subjective: Patient Ava presents with concern ...,4,## Subjective\nThe subjective section provides...,5,## Subjective\nThe subjective section accurate...
1,Good morning. I'm Doctor. Chen. You must be Da...,"Subjective: Patient David, a Marine Corps enli...",4,## Subjective\nThe subjective section captures...,4,## Subjective\nThe subjective section is mostl...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett....",Subjective: Patient Ms. Cooper presents with c...,4,## Subjective\nThe subjective section captures...,4,## Subjective\nThe subjective section is mostl...


Evaluation results for run claude-3.7-sonnet:


,transcript,outputs,completeness/score,completeness/justification,source_grounding/score,source_grounding/justification
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me...",Subjective: Patient Ava presents with concern ...,5,## Subjective\nThe subjective section effectiv...,5,## Subjective\nThe subjective section accurate...
1,Good morning. I'm Doctor. Chen. You must be Da...,"Subjective: Patient David, a Marine Corps enli...",5,## Subjective\nThe subjective section effectiv...,5,## Subjective\nThe subjective section accurate...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett....",Subjective: Patient Ms. Cooper presents with c...,5,## Subjective\nThe subjective section effectiv...,5,## Subjective\nThe subjective section accurate...


Evaluation results for run claude-3.7-sonnet-thinking:


,transcript,outputs,completeness/score,completeness/justification,source_grounding/score,source_grounding/justification
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me...",Subjective: Patient Ava presents with concern ...,4,## Subjective\nThe subjective section is compr...,5,## Subjective\nThe subjective section accurate...
1,Good morning. I'm Doctor. Chen. You must be Da...,"Subjective: Patient David, a Marine Corps enli...",4,## Subjective\nThe subjective section is thoro...,4,## Subjective\nThe subjective section is accur...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett....",Subjective: Patient Ms. Cooper presents with c...,4,## Subjective\nThe subjective section effectiv...,5,## Subjective\nThe subjective section accurate...


Evaluation results for run deepseek-r1:


,transcript,outputs,completeness/score,completeness/justification,source_grounding/score,source_grounding/justification
0,"Hi, Ava. I'm Doctor. Bennett. It's great to me...",Subjective: Patient Ava presents with concern ...,4,## Subjective\nThe note effectively captures t...,5,## Subjective\nThe section accurately captures...
1,Good morning. I'm Doctor. Chen. You must be Da...,"Subjective: Patient David, a Marine Corps enli...",3,## Subjective\nThe note accurately captures Da...,4,## Subjective\nThe section accurately captures...
2,"Good morning, Ms. Cooper. I'm Doctor. Bennett....",Subjective: Patient Ms. Cooper presents with c...,3,## Subjective\nThe note appropriately captures...,4,## Subjective\nThe section accurately captures...
